In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec 22 05:01:58 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!git clone https://github.com/pacman100/DHS-LLM-Workshop.git

Cloning into 'DHS-LLM-Workshop'...
remote: Enumerating objects: 1351, done.
remote: Counting objects: 100% (442/442), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 1351 (delta 311), reused 377 (delta 254), pack-reused 909
Receiving objects: 100% (1351/1351), 45.63 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (743/743), done.


In [3]:
!pip install packaging
!pip uninstall -y ninja && pip install ninja
!ninja --version
!echo $?

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.2 MB/s eta 0:00:00
1.11.1.git.kitware.jobserver-1
0


In [4]:
%cd /content/DHS-LLM-Workshop
!git pull
!pip install -r requirements.txt

/content/DHS-LLM-Workshop
Already up to date.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8gl_me_2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-8gl_me_2
  Resolved https://github.com/huggingface/transformers to commit 3657748b4d1539ca89555f1cf2b465b74f445491
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-gnwe046k
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-gnwe046k
  Resolved https://github.com/huggingface/accelerate to commit b565a6c58a0ef7df041cdd9742914ff9973e15a6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft to /tmp/

In [5]:
!pip install flash-attn --no-build-isolation
!pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.3.6-cp310-cp310-linux_x86_64.whl size=56477261 sha256=652ad256d0891cb2c6d7183f96f7f56ff61cdeee24388381abb35e7a0f2eeca1
  Stored in directory: /root/.cache/pip/wheels/24/5f/16/5044cdddb6dfb3331dfbffa28ab6096ec2900777af5cb0253a
Successfully built flash-attn


In [6]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
%cd chat_assistant/training/

/content/DHS-LLM-Workshop/chat_assistant/training


In [9]:
!git pull
!WANDB_PROJECT=openhathi_instruct python train.py \
--seed 100 \
--model_name "sarvamai/OpenHathi-7B-Hi-v0.1-Base" \
--dataset_name "smangrul/hindi_instruct_v1" \
--chat_template_format "chatml" \
--add_special_tokens False \
--append_concat_token False \
--splits "train,test" \
--max_seq_len 2048 \
--num_train_epochs 3 \
--logging_steps 5 \
--log_level "info" \
--logging_strategy "steps" \
--evaluation_strategy "epoch" \
--save_strategy "epoch" \
--push_to_hub \
--hub_private_repo True \
--hub_strategy "every_save" \
--fp16 True \
--packing True \
--learning_rate 1e-4 \
--lr_scheduler_type "cosine" \
--weight_decay 1e-4 \
--warmup_ratio 0.03 \
--max_grad_norm 1.0 \
--output_dir "OpenHathi-7B-Hi-v0.1-Instruct" \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size 4 \
--gradient_accumulation_steps 4 \
--gradient_checkpointing True \
--use_reentrant False \
--dataset_text_field "content" \
--use_peft_lora True \
--lora_r 8 \
--lora_alpha 16 \
--lora_dropout 0.1 \
--lora_target_modules "q_proj,k_proj,v_proj,o_proj,down_proj,up_proj,gate_proj,embed_tokens,lm_head" \
--use_4bit_qunatization True \
--use_nested_quant True \
--bnb_4bit_compute_dtype "float16" \
--gradient_checkpointing True \
--use_reentrant False \
--use_flash_attn True

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 312 bytes | 312.00 KiB/s, done.
From https://github.com/pacman100/DHS-LLM-Workshop
   91c25c5..6a5b3cf  main       -> origin/main
Updating 91c25c5..6a5b3cf
Fast-forward
 requirements.txt | 1 +
 1 file changed, 1 insertion(+)
2023-12-22 05:07:54.247635: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-22 05:07:54.247691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-22 05:07:54.249427: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attem